# Day 4 - Lab 2: Generating a CI/CD Pipeline

**Objective:** Use an LLM to generate all necessary configuration files to create an automated Continuous Integration (CI) pipeline for the FastAPI application using Docker and GitHub Actions.

**Estimated Time:** 75 minutes

**Introduction:**
A robust CI pipeline is the backbone of modern software development. It automatically builds and tests your code every time a change is made, catching bugs early and ensuring quality. In this lab, you will generate all the configuration-as-code artifacts needed to build a professional CI pipeline for our application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load our application code to provide context for the LLM. The AI needs to see our code's imports to generate an accurate `requirements.txt` file.

**Model Selection:**
Models that are good at understanding code and structured data formats like YAML are ideal. `gpt-4.1`, `o3`, or `codex-mini` are strong choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated configuration files.
- `clean_llm_output()`: To clean up the generated text and code.

In [11]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the application code from Day 3 to provide context
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-10-31 14:50:57,476 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a `requirements.txt`

**Task:** Before we can build a Docker image, we need a list of our Python dependencies. Prompt the LLM to analyze your application code and generate a `requirements.txt` file.

**Instructions:**
1.  Write a prompt that provides the LLM with the source code of your FastAPI application (`app_code`).
2.  Instruct it to analyze the `import` statements and generate a list of all external dependencies (like `fastapi`, `uvicorn`, `sqlalchemy`). You should also ask it to include `pytest` for testing.
3.  The output should be formatted as a standard `requirements.txt` file.
4.  Save the artifact to the project's root directory.

In [23]:
requirements_prompt = f"""
Generate a requirements.txt file for a Python project based on the following application code:

Application Code:
{app_code}

The requirements.txt file should list all necessary Python packages and their versions needed to run the application.
Analyze the code for import statements and generate a list of all external dependencies.
**Include pytest, pydantic, and pydantic[email] for testing.** 

Generate properly formatted requirements.txt content only, without any additional explanations or text.
Do not include specific version numbers for the packages.
**ONLY include the content of the requirements.txt file in your response.
Do not include any extra text, explainations, or formatting. Do not start with "plaintext"**
"""

print("--- Generating requirements.txt ---")
if app_code:
    requirements_content = get_completion(requirements_prompt, client, model_name, api_provider)
    cleaned_reqs = clean_llm_output(requirements_content, language='text')
    print(cleaned_reqs)
    save_artifact(cleaned_reqs, "app/requirements.txt", overwrite=True)
else:
    print("Skipping requirements generation because app code is missing.")

--- Generating requirements.txt ---
fastapi
sqlalchemy
uvicorn
pydantic
pydantic[email]
pytest


### Challenge 2 (Intermediate): Generating a `Dockerfile`

**Task:** Generate a multi-stage `Dockerfile` to create an optimized and secure container image for our application.

> **Tip:** Why a multi-stage Dockerfile? The first stage (the 'builder') installs all dependencies, including build-time tools. The final stage copies only the application code and the necessary installed packages. This results in a much smaller, more secure production image because it doesn't contain any unnecessary build tools.

**Instructions:**
1.  Write a prompt asking for a multi-stage `Dockerfile` for a Python FastAPI application.
2.  Specify the following requirements:
    * Use a slim Python base image (e.g., `python:3.11-slim`).
    * The first stage should install dependencies from `requirements.txt`.
    * The final stage should copy the installed dependencies and the application code.
    * The `CMD` should execute the application using `uvicorn`.
3.  Save the generated file as `Dockerfile` in the project's root.

In [13]:
dockerfile_prompt = f"""
Generate a multi-stage Dockerfile for a Python FastAPI application.

The Dockerfile should follow these specifications:
1. Use a slim Python base image (e.g., `python:3.11-slim`).
2. In the build stage, install dependencies from a requirements.txt file.
3. Copy the installed dependencies and the application code to the final stage.
4. The `CMD` should execute the application using `uvicorn`, assuming the main application file is located at `app/main.py` and the FastAPI app instance is named `app`.

Generate only the Dockerfile content without any additional explanations or text.
"""

print("--- Generating Dockerfile ---")
dockerfile_content = get_completion(dockerfile_prompt, client, model_name, api_provider)
cleaned_dockerfile = clean_llm_output(dockerfile_content, language='dockerfile')

print(cleaned_dockerfile)
save_artifact(cleaned_dockerfile, "app/Dockerfile", overwrite=True)

--- Generating Dockerfile ---
# Use the official Python image as a parent image
FROM python:3.11-slim AS builder

# Set the working directory in the container
WORKDIR /app

# Copy the requirements file into the container
COPY requirements.txt .

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Use a second FROM statement to start a new stage
FROM python:3.11-slim

# Set the working directory in the container
WORKDIR /app

# Copy the dependencies from the builder stage
COPY --from=builder /usr/local/lib/python3.11/site-packages /usr/local/lib/python3.11/site-packages
COPY --from=builder /usr/local/bin /usr/local/bin

# Copy the current directory contents into the container at /app
COPY . .

# Make port 80 available to the world outside this container
EXPOSE 80

# Run app.py when the container launches
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "80"]


WindowsPath('C:/Users/labadmin/Desktop/Workspace/AG-AISOFTDEV/artifacts/app/Dockerfile')

### Challenge 3 (Advanced): Generating the GitHub Actions Workflow

**Task:** Generate a complete GitHub Actions workflow file to automate the build and test process.

**Instructions:**
1.  Write a prompt to generate a GitHub Actions workflow file named `ci.yml`.
2.  Specify the following requirements for the workflow:
    * It should trigger on any `push` to the `main` branch.
    * It should define a single job named `build-and-test` that runs on `ubuntu-latest`.
    * The job should have steps to: 1) Check out the code, 2) Set up a Python environment, 3) Install dependencies from `requirements.txt`, and 4) Run the test suite using `pytest`.
3.  Save the generated YAML file to `.github/workflows/ci.yml`.

In [14]:
ci_workflow_prompt = """
Create a GitHub Actions workflow file (.github/workflows/ci.yml) for a Python FastAPI application.
There should be a single job named 'build-and-test' that runs on 'ubuntu-latest'.

The job should include the following steps:
1. Checkout the repository code.
2. Set up a Python environment with version 3.11.
3. Install dependencies from a requirements.txt file located at the root of the repository.
4. Run tests using pytest, assuming tests are located in a 'tests' directory.

The workflow should be triggered on pushes and pull requests to the main branch.
"""

print("--- Generating GitHub Actions Workflow ---")
ci_workflow_content = get_completion(ci_workflow_prompt, client, model_name, api_provider)
cleaned_yaml = clean_llm_output(ci_workflow_content, language='yaml')
print(cleaned_yaml)

if cleaned_yaml:
    # Note: The save_artifact helper creates directories as needed.
    save_artifact(cleaned_yaml, ".github/workflows/ci.yml", overwrite=True)

--- Generating GitHub Actions Workflow ---
name: CI

on:
  push:
    branches:
      - main
  pull_request:
    branches:
      - main

jobs:
  build-and-test:
    runs-on: ubuntu-latest

    steps:
    - name: Check out repository code
      uses: actions/checkout@v2

    - name: Set up Python 3.11
      uses: actions/setup-python@v2
      with:
        python-version: '3.11'

    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt

    - name: Run tests with pytest
      run: |
        pytest tests


## Lab Conclusion

Excellent! You have now generated a complete, professional Continuous Integration pipeline using AI. You created the dependency list, the containerization configuration, and the automation workflow, all from simple prompts. This is a powerful demonstration of how AI can automate complex DevOps tasks, allowing teams to build and ship software with greater speed and confidence.

> **Key Takeaway:** AI is a powerful tool for generating 'Configuration as Code' artifacts. By prompting for specific formats like `requirements.txt`, `Dockerfile`, or `ci.yml`, you can automate the creation of the files that define your entire build, test, and deployment processes.